# モジュールをインポート

In [241]:
import random
import time

# 計算用の関数

### ネイピア数
$$e = \lim_{{x \to \infty}} \left(1 + \frac{1}{x}\right)^x$$

In [242]:
def napiers_logarithm(x):
    """
    ネイピア数を求める関数

    Args:
        x (float): 入力
    
    Returns:
        float: 出力
    """
    return (1 + 1 / x) ** x
napier_number = napiers_logarithm(100000000)  # e

### 自然対数の近似値

In [243]:
def ln(x, max_iter=20, tol=1e-12):
    """
    自然対数を求める関数

    Args:
        x (float): 入力
        max_iter (int): 最大反復回数
        tol (float): 許容誤差
    
    Returns:
        float: 出力
    """
    if x <= 0: raise ValueError("x must be positive")
    k = 0
    while x > 2:
        x /= 2
        k += 1
    while x < 0.5:
        x *= 2
        k -= 1
    y = x - 1  # ln(1) = 0 付近の値から開始
    for _ in range(max_iter):
        prev_y = y
        y -= (2.718281828459045**y - x) / (2.718281828459045**y)  # f(y) / f'(y)
        if abs(y - prev_y) < tol:
            break
    return y + k * 0.6931471805599453  # ln(2) ≈ 0.693147

### 平方根を計算

In [244]:
def sqrt(x):
    """
    平方根を求める関数

    Args:
        x (float): 入力
    
    Returns:
        float: 出力
    """
    tolerance = 1e-10  # 許容誤差
    estimate = x / 2.0  # 初期推定値
    while True:
        new_estimate = (estimate + x / estimate) / 2  # ニュートン法による更新
        if abs(new_estimate - estimate) < tolerance:  # 収束したら終了
            return new_estimate
        estimate = new_estimate  # 推定値を更新

# 活性化関数およびその微分関数

### シグモイド関数
$$Sigmoid(x) = \frac{1}{1 + e^{-x}}$$
### シグモイド関数の微分
$$Sigmoid'(x) = Sigmoid(x) \cdot (1 - Sigmoid(x))$$

In [245]:
def sigmoid(x):
    """
    Sigmoid 関数

    Args:
        x (float): 入力
    
    Returns:
        float: 出力
    """
    return 1 / (1 + napier_number ** -x)

def sigmoid_derivative(x):
    """
    Sigmoid 関数の微分

    Args:
        x (float): 入力
    
    Returns:
        float: 出力
    """
    return sigmoid(x) * (1 - sigmoid(x))

### ReLU関数
$$ReLU(x) = \max(0, x)$$
### ReLU関数の微分
$$ReLU'(x) = \begin{cases} 
1 & (x > 0) \\
0 & (x ≤ 0)
\end{cases}$$

In [246]:
def relu(x):
    """
    ReLU 関数

    Args:
        x (float): 入力
    
    Returns:
        float: 出力
    """
    return max(0, x)

def relu_derivative(x):
    """
    ReLU 関数の微分

    Args:
        x (float): 入力
    
    Returns:
        float: 出力
    """
    return 1 if x > 0 else 0

### Leaky ReLU関数
$$LeakyReLU(x) = \begin{cases} 
x & (x > 0) \\ 
\alpha x & (x ≤ 0)
\end{cases}$$

### Leaky ReLU関数の微分
$$LeakyReLU'(x) = \begin{cases} 
1 & (x > 0) \\ 
\alpha & (x ≤ 0)
\end{cases}$$

In [247]:
def leaky_relu(x, alpha=0.01):
    """
    Leaky ReLU 関数

    Args:
        x (float): 入力
        alpha (float): ハイパーパラメータ
    
    Returns:
        float: 出力
    """
    return x if x > 0 else alpha * x

def leaky_relu_derivative(x, alpha=0.01):
    """
    Leaky ReLU 関数の微分

    Args:
        x (float): 入力
        alpha (float): ハイパーパラメータ
    
    Returns:
        float: 出力
    """
    return 1 if x > 0 else alpha

### 恒等関数
$$Identity(x) = x$$
### 恒等関数の微分
$$Identity'(x) = 1$$

In [248]:
def identity(x):
    """
    恒等関数

    Args:
        x (float): 入力
    
    Returns:
        float: 出力
    """
    return x

def identity_derivative(x):
    """
    恒等関数の微分

    Args:
        x (float): 入力
    
    Returns:
        float: 出力
    """
    return 1

In [249]:
activation_fuctions = {"sigmoid": sigmoid, "relu": relu, "leaky_relu": leaky_relu, "identity": identity}
derivative_fuctions = {"sigmoid": sigmoid_derivative, "relu": relu_derivative, "leaky_relu": leaky_relu_derivative, "identity": identity_derivative}

# 損失関数

### クロスエントロピー損失
$$ L = -\sum_{i=1}^{N} y_i \cdot \ln(\hat{y}_i + \epsilon)$$

In [250]:
def cross_entropy_loss(y_true, y_pred):
    """
    交差エントロピー損失関数

    Args:
        y_true (list): 正解ラベル
        y_pred (list): 予測ラベル
    
    Returns:
        float: 出力
    """
    if len(y_true) != len(y_pred): raise ValueError("Input lists must have the same length.")
    return -sum([t * ln(p + 1e-9) for t, p in zip(y_true, y_pred)])

### 平均二乗誤差
$$MSE = \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2$$

In [251]:
def mean_squared_error(y_true, y_pred):
    """
    平均二乗誤差を求める関数

    Args:
        y_true (list): 正解ラベル
        y_pred (list): 予測ラベル
    
    Returns:
        float: 出力
    """
    if len(y_true) != len(y_pred): raise ValueError("Input lists must have the same length.")
    return sum([(t - p) ** 2 for t, p in zip(y_true, y_pred)]) / len(y_true)

### 平均絶対誤差
$$MAE = \frac{1}{N} \sum_{i=1}^{N} |y_i - \hat{y}_i|$$

In [252]:
def mean_absolute_error(y_true, y_pred):
    """
    平均絶対誤差を求める関数

    Args:
        y_true (list): 正解ラベル
        y_pred (list): 予測ラベル
    
    Returns:
        float: 出力
    """
    if len(y_true) != len(y_pred): raise ValueError("Input lists must have the same length.")
    return sum([abs(t - p) for t, p in zip(y_true, y_pred)]) / len(y_true)

# ニューラネットワーク

### ニューラルネットワークを初期化

In [253]:
def initialize_weights(layer_sizes):
    """
    重みとバイアスを初期化する関数

    Args:
        layer_sizes (list): 各層のユニット数
    
    Returns:
        tuple: 重みとバイアス
    """
    weights, biases = [], []
    for i in range(len(layer_sizes) - 1):
        limit = sqrt(6 / (layer_sizes[i] + layer_sizes[i+1]))  # 重みの初期化に使う乱数の範囲
        weights.append([[random.uniform(-limit, limit) for _ in range(layer_sizes[i])] for _ in range(layer_sizes[i+1])])  # 重みは -limit から limit の間の乱数で初期化
        biases.append([0 for _ in range(layer_sizes[i+1])])  # バイアスは0で初期化
    return weights, biases

### 順伝播処理

In [254]:
def forward_propagation(inputs, weights, biases, hidden_activation="relu", output_activation="sigmoid"):  # 順伝播処理
    """
    順伝播処理を行う関数

    Args:
        inputs (list): 入力
        weights (list): 重み
        biases (list): バイアス
    
    Returns:
        list: 出力
    """
    activations = [inputs]
    for W, b in zip(weights, biases):
        z = [
            sum([activations[-1][i] * W[j][i] for i in range(len(activations[-1]))]) + b[j]
            for j in range(len(b))
        ]
        if W != weights[-1]:
            activations.append([activation_fuctions[hidden_activation](z_i) for z_i in z])
        else:
            activations.append([activation_fuctions[output_activation](z_i) for z_i in z])
    return activations

### 逆伝播処理

In [255]:
def backward_propagation(activations, y_true, weights, biases, learning_rate, hidden_activation="relu", output_activation="sigmoid"):  # 逆伝播処理
    """
    逆伝播処理を行う関数

    Args:
        activations (list): 出力
        y_true (list): 正解ラベル
        weights (list): 重み
        biases (list): バイアス
        learning_rate (float): 学習率
    
    Returns:
        tuple: 重みとバイアス
    """
    output_layer = activations[-1]
    errors = [
        (output_layer[i] - y_true[i]) * derivative_fuctions[output_activation](output_layer[i])
        for i in range(len(y_true))
    ]
    deltas = [errors]
    # 隠れ層の誤差を計算
    for l in range(len(weights)-1, 0, -1):
        hidden_errors = [
            sum([deltas[0][k] * weights[l][k][j] for k in range(len(deltas[0]))]) * derivative_fuctions[hidden_activation](activations[l][j])
            for j in range(len(activations[l]))
        ]
        deltas.insert(0, hidden_errors)
    # 重みとバイアスを更新
    for l in range(len(weights)):
        for i in range(len(weights[l])):
            for j in range(len(weights[l][i])):
                weights[l][i][j] -= learning_rate * deltas[l][i] * activations[l][j]
            biases[l][i] -= learning_rate * deltas[l][i]

    return weights, biases

### 学習

In [256]:
def train(X, y, layer_sizes, epochs, learning_rate, hidden_activation="relu", output_activation="sigmoid"):  # 学習
    """
    ニューラルネットワークを学習する関数

    Args:
        X (list): 入力
        y (list): 正解ラベル
        layer_sizes (list): 各層のユニット数
        epochs (int): エポック数
        learning_rate (float): 学習率
    
    Returns:
        tuple: 重みとバイアス
    """
    weights, biases = initialize_weights(layer_sizes)
    start = time.time()
    for epoch in range(epochs):
        total_loss = 0
        for i in range(len(X)):
            activations = forward_propagation(X[i], weights, biases, hidden_activation, output_activation)
            total_loss += cross_entropy_loss(y[i], activations[-1])
            weights, biases = backward_propagation(activations, y[i], weights, biases, learning_rate, hidden_activation, output_activation)
        m = epoch // (epochs // 20) + 1
        print(f"\rEpoch {epoch+1}/{epochs}, Loss: {total_loss/len(X):.10f}, [{'+'*m}{' '*(20-m)}]{' '*5}",end="")
    print(f"Training time: {time.time()-start:.2f} seconds")
    return weights, biases

### 予測

In [257]:
def predict(X, weights, biases, hidden_activation="relu", output_activation="sigmoid"):  # 予測
    """
    予測を行う関数

    Args:
        X (list): 入力
        weights (list): 重み
        biases (list): バイアス
    
    Returns:
        list: 出力
    """
    outputs = []
    for i in range(len(X)):  # Prediction
        outputs.append(forward_propagation(X[i], weights, biases, hidden_activation, output_activation)[-1])
    return outputs

### 精度計算

In [258]:
def accuracy(X, y, predict):  # 予測精度の計算
    """
    予測精度を計算する関数

    Args:
        X (list): 入力
        y (list): 正解ラベル
        predict (list): 予測値
    
    Returns:
        float: 予測精度
    """
    accuracy = 0
    total_loss = 0
    for i in range(len(predict)):  # Prediction
        print(f"入力: {X[i]}, 正解: {y[i]}, 予測値: {[0 if p<0.5 else 1 for p in predict[i]]}, 出力地: {predict[i]}")
        accuracy += 1 if [0 if p<0.5 else 1 for p in predict[i]] == y[i] else 0
        total_loss += cross_entropy_loss(y[i], predict[i])
    print(f"正解率: {accuracy / len(predict):.2f}, Loss: {total_loss / len(predict):.10f}")
    return accuracy / len(predict)

### データセット分割

In [259]:
def split_dataset(X, y, train_size=0.8):  # データセットを学習用とテスト用に分割
    """
    データセットを学習用とテスト用に分割する関数

    Args:
        X (list): 入力
        y (list): 正解ラベル
        train_size (float): 学習データの割合
    
    Returns:
        tuple: 学習用データとテスト用データ
    """
    n = len(X)
    indices = list(range(n))
    random.shuffle(indices)
    X_train, y_train = [X[i] for i in indices[:int(n*train_size)]], [y[i] for i in indices[:int(n*train_size)]]
    X_test, y_test = [X[i] for i in indices[int(n*train_size):]], [y[i] for i in indices[int(n*train_size):]]
    return X_train, y_train, X_test, y_test

### メインコード

In [ ]:
# データセット
# X = [[0, 0], [0, 1], [1, 0], [1, 1]]  # 入力
# y = [[1, 1], [1, 0], [0, 1], [0, 0]]  # 出力

X = [[random.random(), random.random()] for _ in range(100)]
y = [[1] if x[0] + x[1] > 1 else [0] for x in X]

X_train, y_train, X_test, y_test = split_dataset(X, y, train_size=0.8)

epochs = 500  # エポック数
learning_rate = 0.01  # 学習率
layer_sizes = [2, 8, 16, 8, 1]  # 各層のユニット数

# X_train: 入力, y_train: 出力, layer_sizes: 各層のユニット数, epochs: エポック数, learning_rate: 学習率
weights, biases = train(X_train, y_train, layer_sizes, epochs, learning_rate, hidden_activation="leaky_relu", output_activation="sigmoid")

# X_test: テスト入力, weights: 重み, biases: バイアス
predict_y = predict(X_test, weights, biases, hidden_activation="leaky_relu", output_activation="sigmoid")

# X_test: テスト入力, y_test: テスト正解ラベル, predict_y: 予測値
accuracy_num = accuracy(X_test, y_test, predict_y)

Epoch 500/500, Loss: 0.0111651133, [++++++++++++++++++++]     Training time: 6.12 seconds
入力: [0.2795043812685317, 0.4789926570128721], 正解: [0], 予測値: [0], 出力地: [3.354308413521764e-05]
入力: [0.15936320396854087, 0.7778978129366259], 正解: [0], 予測値: [0], 出力地: [0.00459474442901708]
入力: [0.3964239144722055, 0.050235567249439006], 正解: [0], 予測値: [0], 出力地: [3.7201398220393824e-06]
入力: [0.00041513041642726733, 0.10568654678366685], 正解: [0], 予測値: [0], 出力地: [3.2532124080827366e-06]
入力: [0.5199764329029976, 0.4296611660742279], 正解: [0], 予測値: [0], 出力地: [0.010746930977614253]
入力: [0.20798535725159906, 0.6562070751464569], 正解: [0], 予測値: [0], 出力地: [0.00031858505868818357]
入力: [0.27474793824914945, 0.9163087109616178], 正解: [1], 予測値: [1], 出力地: [0.9997474019057527]
入力: [0.6045250699646221, 0.2001241925699958], 正解: [0], 予測値: [0], 出力地: [4.81772620585152e-05]
入力: [0.8084413955921818, 0.19005925417885217], 正解: [0], 予測値: [0], 出力地: [0.1210756703716336]
入力: [0.8201467806584752, 0.9087232989882436], 正解: [1], 予測値: 